In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
wandb.init(project="BART-CNN-Convosumm", entity="remeris")

wandb: Currently logged in as: remeris. Use `wandb login --relogin` to force relogin


In [4]:
!pip install peft
!pip install evaluate
!pip install rouge_score

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/82/cc/bf022d6bc3996a5939c3ee39bde2b0e1f8bf6cea6ef9c9cdaf1639586237/peft-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 16.8 MB/s eta 0:00:00
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=8606b30480948b6b4beb4c60e1738cc40d4b9b87f9ec0b64674a6c32b776a00c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
import torch
from transformers import BartTokenizerFast
from transformers import BartConfig
from datasets import Dataset, DatasetDict
from copy import deepcopy
from transformers import BartForConditionalGeneration
import nltk
import numpy as np
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
import evaluate
import random
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
MODELS_PATH = './models/'
DATASETS_PATH = '/kaggle/input/'
DATASET_VERSION = 'arg-filtered/'

In [7]:
def init_random_seed(value=42):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
init_random_seed()

In [8]:
data_dict = {}
for dataset_type in ['train', 'val', 'test']:
    seq_type_dict = {}
    for seq_type in ['source', 'target']:
        with open(DATASETS_PATH + DATASET_VERSION + dataset_type + '.' + seq_type, 'r') as fstream:
            seq_type_dict[seq_type] = fstream.readlines()
    data_dict[dataset_type] = Dataset.from_dict(seq_type_dict)
    
raw_datasets = DatasetDict(data_dict)

tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-large')

MAX_SEQ_LEN = 2048
def preprocess_function(examples, max_length):
    model_inputs = tokenizer(text=examples["source"], max_length=max_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target"], max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

long_tokenized_datasets = raw_datasets.map(lambda x: preprocess_function(x, 2048), batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
def get_longer_model(model):
    cur_state = model.state_dict()
    encoder_pos_embs = cur_state['model.encoder.embed_positions.weight']
    to_append = encoder_pos_embs[2:]
    new_encoder_pos = torch.cat((encoder_pos_embs, to_append))
    cur_state['model.encoder.embed_positions.weight'] = new_encoder_pos
    cur_state['model.decoder.embed_positions.weight'] = new_encoder_pos
    cfg = model.config
    cfg.max_position_embeddings = cfg.max_position_embeddings * 2
    new_model = BartForConditionalGeneration(cfg)
    new_model.load_state_dict(cur_state, strict=True)
    return new_model

In [10]:
metric = evaluate.load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [11]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [12]:
model = get_longer_model(model)

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir='BART-CNN-Convosumm',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True, 
    evaluation_strategy='epoch',
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    label_smoothing_factor=0.1,
    warmup_steps=1,
    lr_scheduler_type='polynomial',
    weight_decay=0.01,
    num_train_epochs=7,
    push_to_hub=True,
    gradient_accumulation_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model='rouge1',
    save_total_limit=1,
    save_strategy='epoch',
    logging_strategy='epoch',
    report_to="wandb",
    hub_model_id='Remeris/BART-CNN-Convosumm',
    hub_strategy='end'
    
)

In [14]:
short_data_collator = DataCollatorForSeq2Seq(tokenizer, model)

trainer = Seq2SeqTrainer(
    tokenizer=tokenizer,
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=long_tokenized_datasets['train'],
    eval_dataset=long_tokenized_datasets['val'],
    data_collator=short_data_collator
)

trainer.train()



You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,6.207000,4.265142,32.334100,7.812000,20.041100,29.484900,77.380000
1,4.024800,3.990273,36.078700,11.044700,21.359600,33.290300,130.580000
2,3.593300,3.901969,34.293100,11.203600,20.793500,30.836100,140.020000
3,3.308600,3.871231,38.484200,11.994700,23.491300,34.434700,85.780000
4,3.112000,3.869993,38.652000,11.831500,23.520800,34.599800,76.200000
5,2.993300,3.880933,38.660000,12.333700,23.439400,35.197600,83.260000
6,2.834000,3.879697,38.625200,12.255600,23.902000,34.632400,81.280000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=70, training_loss=3.7247077396937778, metrics={'train_runtime': 816.046, 'train_samples_per_second': 1.724, 'train_steps_per_second': 0.086, 'total_flos': 1568867334709248.0, 'train_loss': 3.7247077396937778, 'epoch': 6.97})

In [15]:
trainer.predict(long_tokenized_datasets['test'])

PredictionOutput(predictions=array([[2, 0, 0, ..., 1, 1, 1],
       [2, 0, 0, ..., 1, 1, 1],
       [2, 0, 0, ..., 1, 1, 1],
       ...,
       [2, 0, 0, ..., 1, 1, 1],
       [2, 0, 0, ..., 1, 1, 1],
       [2, 0, 0, ..., 1, 1, 1]]), label_ids=array([[    0, 45331,   268, ...,     1,     1,     1],
       [    0, 29182, 22800, ...,     1,     1,     1],
       [    0,  2895, 40471, ...,     1,     1,     1],
       ...,
       [    0, 10787,  2003, ...,     1,     1,     1],
       [    0,   133, 40471, ...,     1,     1,     1],
       [    0,  2895, 40471, ...,     1,     1,     1]]), metrics={'test_loss': 3.834308624267578, 'test_rouge1': 38.3642, 'test_rouge2': 12.2056, 'test_rougeL': 23.7782, 'test_rougeLsum': 34.3959, 'test_gen_len': 84.132, 'test_runtime': 315.5988, 'test_samples_per_second': 0.792, 'test_steps_per_second': 0.792})

In [16]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/Remeris/BART-CNN-Convosumm/tree/main/'

In [17]:
translated_source = """
I personally don’t know how to resist shootings, if only to install a metal detector, because the cleaning lady Baba Zina is unlikely to cope alone It’s hard to believe, but a person needs to be brought to this level. good psychologists are needed at school, team building class when a person gets to know the people around him better, he begins to understand what and why, their characters, begins to make friends and finds common topics for communication if this is not the case, a person who is uncommunicative ends up in a kind of bubble, because he himself cannot start communication, and perhaps he doesn’t want to, because he thinks that they are different although children are essentially very similar, they are simply formed in the same society, the only difference between them is their parents and hobby groups I’m not sure that’s the problem. I gave a fuck about everyone at school. But there was one in my class who was very close to shooting everyone and everything if he had a gun at hand. but not because he was quiet I agree about psychologists here. if we talk about school shooting, it’s always a combination of factors there is no school shooting just because of parents or just because of school a person, whatever one may say, is a social creature, wants to chat, share joy, and maybe grief. When there is no one and nowhere to share emotions, it becomes difficult plus children are always prone to extremes. As a child, I couldn’t even think that grades in quarters would never be useful in life. I thought that somehow they would move on. I can’t even imagine that in principle you don’t have to go to school, just pass the exams and learn the topic from the textbook and everything will be okay but everyone told us that you should teach, otherwise you’ll go to a workhouse, or sweep the street, so-so motivation for an intimidated child apparently I was the wrong child. """

In [18]:
inf_input = tokenizer(translated_source, max_length=2048, return_tensors="pt", truncation=True)
model_output = model.generate(input_ids=inf_input.input_ids.cuda())
tokenizer.batch_decode(model_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Most of the commenters agree that psychologists are needed at school to help students cope with social situations such as school shootings. A few commenters say that they did not know how to resist shootings. One commenter says that a person needs to be more social, while another says that it’s a combination of factors, with one commenter saying that children are very similar to each other.\n'